In [6]:
### Парсер с вакансий с сайта HH.RU

API hh.ru ограничивает запросы до 2000 записей на один фильтр, поэтому перебираем регионы и текстовые запросы, чтобы увеличить объем данных.
Я беру города-миллионники России за короткий период (2-3 месяца)
Расширить справочник можно добавив новые города areas API hh.ru

In [1]:
import requests
import json
import time
import os
from datetime import datetime, timedelta

def getPages(page=0, area=1, date_from=None, date_to=None):
    """Функция для получения страницы вакансий."""
    params = {
        'text': 'data engineer',  # Поисковый запрос (можно адаптировать)
        'area': area,            # Регион
        'page': page,            # Номер страницы
        'per_page': 100,         # Количество вакансий на странице
        'date_from': date_from,  # Дата начала
        'date_to': date_to       # Дата окончания
    }

    response = requests.get('https://api.hh.ru/vacancies', params=params)
    if response.status_code != 200:
        print(f"Ошибка: {response.status_code}")
        return None

    return response.json()

# Города-миллионники России (ID регионов из API hh.ru)
cities = [1, 2, 1207, 4, 5, 6, 7, 10, 13, 99, 72]  # Москва, СПб, Новосибирск и другие

# Даты за последние 2–3 месяца
date_to = datetime.now().strftime('%Y-%m-%d')
date_from = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')

# Папка для сохранения данных
output_dir = '/content/parser_hh'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_count = 0
for city in cities:
    for page in range(0, 20):  # 20 страниц по 100 записей = 2000 записей на город
        data = getPages(page=page, area=city, date_from=date_from, date_to=date_to)
        if data is None or 'items' not in data:
            break

        # Сохраняем данные
        next_file_name = os.path.join(output_dir, f'{file_count}.json')
        with open(next_file_name, mode='w', encoding='utf8') as file:
            json.dump(data['items'], file, ensure_ascii=False, indent=3)

        file_count += 1
        print(f"Сохранен файл: {next_file_name} (Город ID: {city}, Страница: {page})")

        # Проверяем, достигнута ли последняя страница
        if data['pages'] - page <= 1:
            break

        time.sleep(0.25)

print(f'Собрано {file_count * 100} записей!')


Сохранен файл: /content/parser_hh/0.json (Город ID: 1, Страница: 0)
Сохранен файл: /content/parser_hh/1.json (Город ID: 1, Страница: 1)
Сохранен файл: /content/parser_hh/2.json (Город ID: 1, Страница: 2)
Сохранен файл: /content/parser_hh/3.json (Город ID: 1, Страница: 3)
Сохранен файл: /content/parser_hh/4.json (Город ID: 1, Страница: 4)
Сохранен файл: /content/parser_hh/5.json (Город ID: 1, Страница: 5)
Сохранен файл: /content/parser_hh/6.json (Город ID: 1, Страница: 6)
Сохранен файл: /content/parser_hh/7.json (Город ID: 1, Страница: 7)
Сохранен файл: /content/parser_hh/8.json (Город ID: 2, Страница: 0)
Сохранен файл: /content/parser_hh/9.json (Город ID: 2, Страница: 1)
Сохранен файл: /content/parser_hh/10.json (Город ID: 1207, Страница: 0)
Сохранен файл: /content/parser_hh/11.json (Город ID: 4, Страница: 0)
Сохранен файл: /content/parser_hh/12.json (Город ID: 5, Страница: 0)
Сохранен файл: /content/parser_hh/13.json (Город ID: 6, Страница: 0)
Сохранен файл: /content/parser_hh/14.json

In [2]:
import os
import json

output_dir = '/content/parser_hh'
all_data = []

# Объединяем данные из всех файлов
for file_name in os.listdir(output_dir):
    file_path = os.path.join(output_dir, file_name)
    with open(file_path, 'r', encoding='utf8') as file:
        data = json.load(file)
        all_data.extend(data)

# Сохраняем объединенные данные
with open('/content/all_vacancies.json', 'w', encoding='utf8') as file:
    json.dump(all_data, file, ensure_ascii=False, indent=3)

print(f'Объединено {len(all_data)} записей в all_vacancies.json')


Объединено 915 записей в all_vacancies.json


In [3]:
from google.colab import files
files.download('/content/all_vacancies.json')
#Скачивание итогового датасета

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>